<div >
<img src = "figs/dsa_banner.png" />
</div>

# Preprocesamiento

### Integrantes:
    - Catalina García García
    - Camilo Alejandro Grande Sánchez
    - Jesús Alberto Parada Pérez
    - Jorge Esteban Caballero Rodríguez


In [4]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

### Importar datos

In [5]:
#importar conjunto de datos
def cargar_datos(filename, nombre_base):
    data = pd.read_excel(f"data/{filename}",index_col=0)
    return data

data = cargar_datos("BD_G22_P.xlsx", "Base de datos:")    

#separar X y y
X=data.iloc[:, 1:11]
y=data['Puntaje_General_Estandarizado']

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### MLFLOW

In [6]:
# defina el servidor para llevar el registro de modelos y artefactos
mlflow.set_tracking_uri('http://localhost:8080')
# registre el experimento
experiment = mlflow.set_experiment("Random_Forest_Model_Experiments")

# Definir la grilla de hiperparámetros a probar para el RandomForestRegressor

n_estimators= [100, 200, 300, 400, 500]
max_depths= [None, 5, 10, 15]
min_samples_splits= [2, 5, 10]
min_samples_leafs= [1, 2, 4]

for n_estimator in n_estimators:
    for max_depth in max_depths:
        for min_samples_split in min_samples_splits:
            for min_samples_leaf in min_samples_leafs:
                with mlflow.start_run(experiment_id=experiment.experiment_id,
                                     run_name =f"random-forest-model-num_trees{n_estimator}-maxdepth{max_depth}-ssplit{min_samples_split}-sleaf{min_samples_leaf}"):

            
                    # Crear el modelo de RandomForestRegressor
                    rf = RandomForestRegressor(n_estimators = n_estimator,
                                               max_depth = max_depth,
                                              min_samples_split  = min_samples_split,
                                              min_samples_leaf = min_samples_leaf)
                    rf.fit(X_train, y_train)

                    predictions = rf.predict(X_test)


                        # Registre los parámetros
                    mlflow.log_param("num_trees", n_estimator)
                    mlflow.log_param("maxdepth", max_depth)
                    mlflow.log_param("min_samples_split", min_samples_split)
                    mlflow.log_param("min_samples_leaf", min_samples_leaf)

                    mlflow.sklearn.log_model(rf, f"random-forest-model-num_trees{n_estimator}-maxdepth{max_depth}-ssplit{min_samples_split}-sleaf{min_samples_leaf}")

                    mse = mean_squared_error(y_test, predictions)
                    mlflow.log_metric("mse", mse)
                    print(mse)
                    mlflow.end_run()

MlflowException: API request to http://localhost:8080/api/2.0/mlflow-artifacts/artifacts/472698015293778482/6ceead1906b14fffa8e10f8430e750b3/artifacts/random-forest-model-num_trees100-maxdepthNone-ssplit2-sleaf1/conda.yaml failed with exception HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/472698015293778482/6ceead1906b14fffa8e10f8430e750b3/artifacts/random-forest-model-num_trees100-maxdepthNone-ssplit2-sleaf1/conda.yaml (Caused by ResponseError('too many 500 error responses'))